In [2]:
import rasterio 
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
from rasterio import features
import os
import xarray as xr
import fnmatch
import time
import multiprocessing as mp 
from glob import glob
from multiprocessing import Pool

In [4]:
DATA_IN = '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/'
# Always use keep the same 
DATA_INTERIM = '/home/cascade/projects/UrbanHeat/data/interim/'

In [6]:
# Open the file with GeoPANDAS read_file
ghs_ids_fn = 'GHS-UCSB-IDS.csv'
ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)

# Open Polygon Raster
polyRst_fn = 'GHS_UCDB_Raster_touched.tif'
polyRst = rasterio.open(DATA_INTERIM+polyRst_fn)

# Turn polyRst data as Xarray, 
polyRst_da = xr.DataArray(polyRst.read(1), dims = ['y', 'x'])

In [7]:
# Get dir list
dir_list = sorted(glob(DATA_IN+'*/'))
dir_list



['/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1984/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1985/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1986/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1987/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1988/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1989/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1990/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1991/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1992/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1993/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1994/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1995/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1996/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1997/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1998/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1999/',
 '/home/CHIRTS

In [8]:
# For ERA5 Dir 
dir_list = dir_list[:-1]
dir_list

['/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1984/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1985/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1986/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1987/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1988/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1989/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1990/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1991/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1992/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1993/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1994/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1995/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1996/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1997/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1998/',
 '/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1999/',
 '/home/CHIRTS

In [10]:
rst = rasterio.open(dir_list[0]+'RH.1983.01.01.tif')

In [14]:
test = rst.read(1)

In [15]:
test[np.isnan(test)] = -9999
test

array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)

In [13]:
polyRst.read(1)

array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)

In [22]:
tempRst_da = xr.DataArray(rst.read(1), dims = ['y', 'x']) # y and x are our 2-d labels
tempRst_da.data[np.isnan(tempRst_da.data)] = -9999
tempRst_da


<xarray.DataArray (y: 2600, x: 7200)>
array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]], dtype=float32)
Dimensions without coordinates: y, x

In [24]:
polyRst_da = xr.DataArray(polyRst.read(1), dims = ['y', 'x'])
polyRst_da

<xarray.DataArray (y: 2600, x: 7200)>
array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]], dtype=float32)
Dimensions without coordinates: y, x

In [25]:
# Make xarray dataset
ds = xr.Dataset(data_vars = 
        {'ghs' : (['y', 'x'], polyRst_da),
        'temp' : (['y', 'x'], tempRst_da),})

In [26]:
ds

<xarray.Dataset>
Dimensions:  (x: 7200, y: 2600)
Dimensions without coordinates: x, y
Data variables:
    ghs      (y, x) float32 -9999.0 -9999.0 -9999.0 ... -9999.0 -9999.0 -9999.0
    temp     (y, x) float32 -9999.0 -9999.0 -9999.0 ... -9999.0 -9999.0 -9999.0

In [30]:
ds_mask = ds.where(ds.temp != -9999, drop = False)
ds_mask = ds_mask.where(ds_mask.ghs > 0, drop = False)

In [31]:
avg = ds_mask.groupby('ghs').mean(xr.ALL_DIMS)

In [32]:
avg

<xarray.Dataset>
Dimensions:  (ghs: 13067)
Coordinates:
  * ghs      (ghs) float64 1.0 3.0 4.0 5.0 ... 1.313e+04 1.313e+04 1.314e+04
Data variables:
    temp     (ghs) float32 64.95055 51.173065 56.16901 ... 66.358116 67.7078

In [42]:
avg_2 = ds_mask.groupby('ghs').mean(xr.ALL_DIMS, skipna = True)

In [43]:
avg_2

<xarray.Dataset>
Dimensions:  (ghs: 13067)
Coordinates:
  * ghs      (ghs) float64 1.0 3.0 4.0 5.0 ... 1.313e+04 1.313e+04 1.314e+04
Data variables:
    temp     (ghs) float32 64.95055 51.173065 56.16901 ... 66.358116 67.7078

In [49]:
dir96 = dir_list[13]

In [50]:
for fn in sorted(os.listdir(dir96)):
    print(fn)

RH.1996.01.01.tif
RH.1996.01.02.tif
RH.1996.01.03.tif
RH.1996.01.04.tif
RH.1996.01.05.tif
RH.1996.01.06.tif
RH.1996.01.07.tif
RH.1996.01.08.tif
RH.1996.01.09.tif
RH.1996.01.10.tif
RH.1996.01.11.tif
RH.1996.01.12.tif
RH.1996.01.13.tif
RH.1996.01.14.tif
RH.1996.01.15.tif
RH.1996.01.16.tif
RH.1996.01.17.tif
RH.1996.01.18.tif
RH.1996.01.19.tif
RH.1996.01.20.tif
RH.1996.01.21.tif
RH.1996.01.22.tif
RH.1996.01.23.tif
RH.1996.01.24.tif
RH.1996.01.25.tif
RH.1996.01.26.tif
RH.1996.01.27.tif
RH.1996.01.28.tif
RH.1996.01.29.tif
RH.1996.01.30.tif
RH.1996.01.31.tif
RH.1996.02.01.tif
RH.1996.02.02.tif
RH.1996.02.03.tif
RH.1996.02.04.tif
RH.1996.02.05.tif
RH.1996.02.06.tif
RH.1996.02.07.tif
RH.1996.02.08.tif
RH.1996.02.09.tif
RH.1996.02.10.tif
RH.1996.02.11.tif
RH.1996.02.12.tif
RH.1996.02.13.tif
RH.1996.02.14.tif
RH.1996.02.15.tif
RH.1996.02.16.tif
RH.1996.02.17.tif
RH.1996.02.18.tif
RH.1996.02.19.tif
RH.1996.02.20.tif
RH.1996.02.21.tif
RH.1996.02.22.tif
RH.1996.02.23.tif
RH.1996.02.24.tif
RH.1996.02

In [69]:
# DATA: This is the data we are using, as string ---<<<<< ALWAYS UPDATE CPT 2020.03.25
DATA = 'RH.' #'Tmax.'
FN_OUT = 'GHS-ERA5-RH' #Tmax'
# FN_OUT = 'GHS-Tmax-DAILY'     #<<<------------    ALWAYS UPDATE
# FN_OUT = 'GHS-Tmax-RH'     #<<<------------    ALWAYS UPDATE

# Loop through dirs in //
def temp_ghs(dir_nm):
    
    # Start Loop
    for fn in sorted(os.listdir(dir_nm)):
        
        # Set dir name for writing files
        dir_year = dir_nm.split(DATA_IN)[1].split('/')[0]
        
        tempRst = dir_nm+fn
        print(dir_nm)
        print(tempRst)
        tempRst = rasterio.open(dir_nm+'/'+fn)
        print(dir_nm+'/'+fn)
        print(tempRst.meta)

        # find all the tif files
        if fn.startswith(DATA):        

            # Get the date of each chirt file
            date = (fn.split(DATA)[1].split('.tif')[0]) 
            print(dir_year)
            print(date, '\n')

    print('DONE ! ! !')

In [70]:
temp_ghs(dir_list[0])

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.01.01.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.01.01.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.01.01 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.01.02.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.01.02.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.01.02 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.01.03.tif
/

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.03.12.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.03.12 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.03.13.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.03.13.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.03.13 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.03.14.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.03.14.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.05.16.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.05.16.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.05.16 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.05.17.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.05.17.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.05.17 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.05.18.tif
/

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.07.23.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.07.23 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.07.24.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.07.24.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.07.24 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.07.25.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.07.25.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None

1983.10.08 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.10.09.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.10.09.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.10.09 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.10.10.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.10.10.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.10.10 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.198

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.12.23.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.12.23 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.12.24.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.12.24.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7200, 'height': 2600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.05000000074505806, 0.0, -180.0,
       0.0, -0.05000000074505806, 70.0)}
1983
1983.12.24 

/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983/RH.1983.12.25.tif
/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1983//RH.1983.12.25.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None

In [76]:
rasterio.open('////home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum/1996/RH.1996.07.22.tif').read(1)

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [80]:
#### DIR PATHS & Arguments
DIR_Tmax = '/home/cascade/projects/UrbanHeat/data/interim/ERA5_Tmax/'
DIR_RH = '/home/cascade/projects/UrbanHeat/data/interim/ERA5_RH/'
DIR_HI = '/home/cascade/projects/UrbanHeat/data/interim/ERA5_HI/'
unit_in = 'C'
unit_out = 'C'

FN_RH_IN = 'GHS-ERA5-RH_' # name of RH file in
FN_Tmax_IN = 'GHS-ERA5-Tmax_' # name of Tmax file in
FN_OUT = 'GHS-ERA5-HI_' # name for files out

In [83]:
Tmax_fn_list = glob(DIR_Tmax+'*.csv')
RH_fn_list = glob(DIR_RH+'*.csv')

for Tmax_fn, RH_fn in zip(sorted(Tmax_fn_list),sorted(RH_fn_list)):
    # Check the years RH and Tmax 
    Tmax_year = Tmax_fn.split(FN_Tmax_IN)[1].split('.csv')[0] 
    print('Tmax year is ', Tmax_year)
    RH_year = RH_fn.split(FN_RH_IN)[1].split('.csv')[0] 
    print('RH year is ', RH_year)

Tmax year is  1983
RH year is  1983
Tmax year is  1984
RH year is  1984
Tmax year is  1985
RH year is  1985
Tmax year is  1986
RH year is  1986
Tmax year is  1987
RH year is  1987
Tmax year is  1988
RH year is  1988
Tmax year is  1989
RH year is  1989
Tmax year is  1990
RH year is  1990
Tmax year is  1991
RH year is  1991
Tmax year is  1992
RH year is  1992
Tmax year is  1993
RH year is  1993
Tmax year is  1994
RH year is  1994
Tmax year is  1995
RH year is  1995
Tmax year is  1996
RH year is  1996
Tmax year is  1997
RH year is  1997
Tmax year is  1998
RH year is  1998
Tmax year is  1999
RH year is  1999
Tmax year is  2000
RH year is  2000
Tmax year is  2001
RH year is  2001
Tmax year is  2002
RH year is  2002
Tmax year is  2003
RH year is  2003
Tmax year is  2004
RH year is  2004
Tmax year is  2005
RH year is  2005
Tmax year is  2006
RH year is  2006
Tmax year is  2007
RH year is  2007
Tmax year is  2008
RH year is  2008
Tmax year is  2009
RH year is  2009
Tmax year is  2010
RH year i

In [9]:
pdays = pd.read_csv('/home/cascade/projects/UrbanHeat/data/processed/AllDATA-GHS-ERA5-HI406-PDAYS.csv')
fig = pd.read_csv('/home/cascade/projects/UrbanHeat/data/processed/AllDATA-GHS-ERA5-HI406-FIGDATA.csv')

In [10]:
len(fig)

391816

In [11]:
len(pdays)

390524